# PIMALUOS Training Visualization

This notebook visualizes the training process for the GNN model,
including pre-training and physics-informed training phases.

In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Results

In [ ]:
# Load results from demo
results_path = Path('../results/small_scale_demo/results.json')

with open(results_path) as f:
    results = json.load(f)

print("Configuration:")
for key, value in results['config'].items():
    print(f"  {key}: {value}")

print("\nData:")
for key, value in results['data'].items():
    print(f"  {key}: {value}")

## Training Curves

Visualize GNN pre-training and physics-informed training losses.

In [ ]:
# Note: Training losses are in the checkpoint, not results.json
# Load checkpoint to get full training history

import torch

checkpoint = torch.load('../results/small_scale_demo/checkpoint.pth', map_location='cpu')
history = checkpoint['training_history']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pre-training losses
ax1.plot(history['pretrain_losses'], linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('GNN Pre-training', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Physics-informed losses
ax2.plot(history['physics_losses'], linewidth=2, color='orangered')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss (with Physics Penalty)', fontsize=12)
ax2.set_title('Physics-Informed Training', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/small_scale_demo/training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Pre-training: {len(history['pretrain_losses'])} epochs")
print(f"  Initial loss: {history['pretrain_losses'][0]:.4f}")
print(f"  Final loss: {history['pretrain_losses'][-1]:.4f}")
print(f"\nPhysics training: {len(history['physics_losses'])} epochs")
print(f"  Initial loss: {history['physics_losses'][0]:.4f}")
print(f"  Final loss: {history['physics_losses'][-1]:.4f}")

## Performance Summary

In [ ]:
print("Performance:")
print(f"  Total time: {results['performance']['total_time_formatted']}")
print(f"\nAction Distribution:")
actions = results['actions']
total = sum(actions.values())
for action, count in actions.items():
    pct = 100 * count / total
    print(f"  {action}: {count} ({pct:.1f}%)")

In [ ]:
# Action distribution pie chart
fig, ax = plt.subplots(figsize=(8, 8))

actions = results['actions']
labels = ['Decrease FAR', 'Maintain FAR', 'Increase FAR']
values = [actions['decrease_far'], actions['maintain_far'], actions['increase_far']]
colors = ['#e74c3c', '#95a5a6', '#3498db']

ax.pie(values, labels=labels, colors=colors, autopct='%1.1f%%', 
       startangle=90, textprops={'fontsize': 12})
ax.set_title('Final Land-Use Actions', fontsize=16, fontweight='bold')

plt.savefig('../results/small_scale_demo/action_distribution.png', dpi=300, bbox_inches='tight')
plt.show()